# Neural networks - Single neuron as Binary Classifier (Worked Example)

<hr style="border:2px solid gray">

# Index: <a id='index'></a>
1. [Setting Up Our Data](#data)
1. [Activation Function](#activation)
1. [Building a Neuron](#neuron)
1. [Building a Training Function](#trainfunc)
1. [Training](#training)
1. [Visualise What We've Done](#visual)

<hr style="border:2px solid gray">

# Setting Up Our Data [^](#index) <a id='data'></a> 

We begin by importing the relevant modules and generating a toy dataset.

In [ ]:
import sklearn, sklearn.datasets
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
np.random.seed(2) # This seed seems to get a nice gently overlapping dataset
n_samples=200

# Toy dataset with sklearn

X, Y = sklearn.datasets.make_classification(n_features=2, n_redundant=0, n_samples=n_samples,
    n_informative=2, random_state=None, n_clusters_per_class=1)

In [ ]:
plt.rcParams['figure.figsize'] = [3,3] # 3x3 inch is good for a laptop screen; 5x5 for external monitor
plt.rcParams['figure.dpi'] = 200 

colors = sns.color_palette("tab10", as_cmap=True)
plt.scatter(X[:,0],X[:,1], c = Y, s=20, marker = 'x', cmap = colors)
plt.show()

<hr style="border:2px solid gray">

# Activation Function [^](#index) <a id='activation'></a>

Thus far in the course we have covered classical machine learning algorithms, most of which make use of linear combinations of the data fed to them. The scope of what we are able to achieve is widened when we add non-linearity to our algorithms, using activation functions.

As you will discover, neurons in a neural network are arranged in layers. Not unlike in the human brain, a neuron receives signals of varying strength from other neurons, and essentially 'decides' whether this combined received signal is strong enough for the neuron to 'fire', and how strongly.

This 'decision' element is replicated by the presence of the activation function in our neuron. All contributions from previous neurons to which it is connected are summed, and passed to the activation function, which then influences how strong a signal our neuron outputs, if any at all.

Without this non-linear aspect to our neuron, it would simply output a linear combination of the data passed to it by previous neurons, which in turn also simply contain some linear combination of the data passed to them. We would find that the output of our algorithm is nothing but a linear combination of our input data, and we have therefore achieved nothing special. 

The real prediction power of neural networks stems precisely from the non-linearity brought about by the presence of activation functions and, as you will find out, there are several choices of activation function we can make, each of which have their own advantages and drawbacks. 

<div style="background-color:#C2F5DD">
    
One of the most common activation functions is the sigmoid function. Make your own below.

In [ ]:
def sigmoid(v):
    s = 1 / (1+np.exp(-v))
    return s

In [ ]:
# plot your Sigmoid function 
x=np.linspace(-10,10)
plt.plot(x,sigmoid(x))

You can run the code cell below to perform some basic checks that your sigmoid function is correct.

In [ ]:
assert sigmoid(0.0) == 0.5 # zero of sigmoid should be 0.5
assert sigmoid(10.0) - 0.9999 < 0.0005
assert sigmoid(-10.0) < 0.0005
# Does this run? Your sigmoid is hopefully OK!

<hr style="border:2px solid gray">

# Building a Neuron [^](#index) <a id='neuron'></a>

<img src="https://static.packt-cdn.com/products/9781788397872/graphics/bc193cf1-aeb4-432e-9f21-e86c1fd45160.png" width="450" height="300" />

A single neuron consists of weight(s), a bias term (normally but not in this
case) and an activation function.
Data which is fed into the neuron is multiplied by the weight (dot product).
The result of this computation is passed through the activation function and output by the neuron.

<div style="background-color:#C2F5DD">

Build your own neuron below.

In [ ]:
def neuron(X, w=[0.0,0.0]):
    a=np.matmul(X,w) # product of data and the weights vector
    y=sigmoid(a)     # activation function for non-linearity
    return y

In [ ]:
assert neuron([0.0], w=[1.0]) == 0.5 # one weight of 1.0 = a neuron that is the sigmoid function
assert neuron([0.5,0.5], w=[0.0,0.0])==0.5

In [ ]:
# Let's plot the neuron
Xrange=np.linspace(-5,5,100)

plt.plot(Xrange,neuron(np.transpose([Xrange]),w=[1.5])) # edit the weight to see what happens!

<hr style="border:2px solid gray">

# Building a Training Function [^](#index) <a id='trainfunc'></a>

Training your model works using the familiar gradient descent method, hence we require some gradients: that is, the rate of change of the loss function with respect to each parameter of the model.

In [ ]:
eta=0.1 # learning rate
alpha=0.0 # decay parameter

In [ ]:
# weight matrix should look something like...
w=np.array([0.5,0.5])

<div style="background-color:#C2F5DD">

Fill out variable a. a is what is passed into the activation function in the equation below.

$$y=\sigma ( X*w ) $$

where $*$ represents matrix multiplication here.

In [ ]:
a=np.matmul(X,w)
np.shape(a) # it's always useful to keep track of the shape of your 'tensors' as you progress through the code

<div style="background-color:#C2F5DD">

We then need to pass this to our non-linear function, in this case the sigmoid function.

In [ ]:
y=sigmoid(a)

<div style="background-color:#C2F5DD">

We need to define an error - here we use the absolute difference (L1 norm) between prediction and observation, given by $e$.

$$ e=|Y-y| $$

In [ ]:
e=Y-y

The formula for the gradient is then:

$$ gradient = - X^T*e $$

<div style="background-color:#C2F5DD"

Implement this below.

In [ ]:
gradient = - np.matmul(np.transpose(X),e)
gradient

Finally we update our weights.

In [ ]:
w= w - eta * (gradient + alpha * w)

<hr style="border:2px solid gray">

# Training [^](#index) <a id='training'></a>    

In the cells above, we completed one training step by computing dot products with the input data and the model weights, then passing to the activation function, calculating the error and updating our parameter values using the gradient. Much more convenient would be to have a function which completes one full training step.

<div style="background-color:#C2F5DD">


<div style="background-color:#C2F5DD"

Implement your training function below.

In [ ]:
def train(X,Y,w, eta=0.02, alpha=0.0):
    a=np.matmul(X,w)
    y=sigmoid(a)
    e=Y-y
    gradient= -np.matmul(np.transpose(X),e)
    w=w-eta*(gradient+alpha*w)
    loss=sum(abs(e)) # overall absolute loss
    return(w,loss)

<hr style="border:2px solid gray">

## Training loop

We can now use our train function in a `for` loop.

<div style="background-color:#C2F5DD"

Experiment with different values of the eta and alpha hyperparameters and observe the effect on the training process using the next section.

In [ ]:
w=np.array([0.1, 0.1]) # initial neuron weights

weights=[]
loss=[]
for i in range(1,100): # run this many training steps
    w,e=train(X,Y,w, eta=0.02, alpha=0.5)   # train
    weights.append(w)  # keep track of the weights
    loss.append(e)     # keep track of the loss

<hr style="border:2px solid gray">

# Visualise What We've Done [^](#index) <a id='visual'></a> 

In [ ]:
# Plot neuron weights, and loss on the same axis, versus training epoch
plt.figure(1)
plt.subplot(211)

plt.plot(weights)
plt.xlabel("Training epoch")
plt.ylabel("Neuron weights")

plt.subplot(212)

plt.plot(loss)
plt.xlabel("Training epoch")
plt.ylabel("Loss")

In [ ]:
# Final loss - this is the thing to minimise!
loss[-1]

In [ ]:
# make a grid of predictions, for plotting the decision boundary
N=100
Xgrid=np.meshgrid(np.linspace(-5, 5, N), np.linspace(-5, 5, N))
Xgrid2=np.array([np.ndarray.flatten(Xgrid[0]), np.ndarray.flatten(Xgrid[1])])
predict=neuron(np.transpose(Xgrid2),w) # re-using our neuron function from earlier
predict=predict.reshape( (N,N) )

In [ ]:
# filled contour of the decision boundary
plt.contourf(Xgrid[0], Xgrid[1] ,predict, cmap=plt.cm.Spectral, alpha=0.8)
# scatter plot of the training data
plt.scatter(X[:,0],X[:,1], c=Y, s=10, cmap=plt.cm.Spectral)

In [ ]:
# This function visualises the neural decision boundary as you train
w=np.array([0.1, 0.1]) #initial neuron weights

for i in range(1,26):
    ax = plt.subplot(5, 5, i)
    plt.axis('off')
    
    
    N=25
    Xgrid=np.meshgrid(np.linspace(-5, 5, N), np.linspace(-5, 5, N))
    Xgrid2=np.array([np.ndarray.flatten(Xgrid[0]), np.ndarray.flatten(Xgrid[1])])
    predict=neuron(np.transpose(Xgrid2),w) # re-using our neuron function from earlier
    predict=predict.reshape( (N,N) )
    
    plt.contourf(Xgrid[0], Xgrid[1] ,predict, cmap=plt.cm.Spectral, alpha=0.8)
    # scatter plot of the training data
    plt.scatter(X[:,0],X[:,1], c=Y, s=1, cmap=plt.cm.Spectral)
    
    w,loss=train(X,Y,w)
